In [1]:
#importing necessary libraries
import pandas as pd
import numpy as np
import warnings                   # To ignore the warnings
warnings.filterwarnings("ignore")
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/final-data/train_final.csv
/kaggle/input/final-data/test_final.csv


In [2]:
# Reading the test data files
train_final_merged = pd.read_csv('/kaggle/input/final-data/train_final.csv')
test_final_merged = pd.read_csv('/kaggle/input/final-data/test_final.csv')

In [3]:
# checking the final merged test dataset
test_final_merged.head()

,Unnamed: 0,farm_id,ingredient_type,id,operations_commencing_year,num_processing_plants,farm_area,farming_company,deidentified_location,temp_obs,cloudiness,wind_direction,pressure_sea_level,precipitation,wind_speed,month,day,hour
0,0,fid_110884,ing_w,0,new,7,690.455096,Obery Farms,location 7369,17.8,4.0,100.0,1021.4,0.0,3.6,1,1,0
1,1,fid_90053,ing_w,1,new,7,252.696160,Obery Farms,location 7369,17.8,4.0,100.0,1021.4,0.0,3.6,1,1,0
2,2,fid_17537,ing_w,2,moderate,7,499.446528,Obery Farms,location 7369,17.8,4.0,100.0,1021.4,0.0,3.6,1,1,0
3,3,fid_110392,ing_w,3,moderate,7,2200.407555,Obery Farms,location 7369,17.8,4.0,100.0,1021.4,0.0,3.6,1,1,0
4,4,fid_62402,ing_w,4,moderate,7,10833.140121,Obery Farms,location 7369,17.8,4.0,100.0,1021.4,0.0,3.6,1,1,0


In [4]:
# checking the  data types of final merged tarin dataset
train_final_merged.dtypes

Unnamed: 0                      int64
farm_id                        object
ingredient_type                object
yield                         float64
operations_commencing_year     object
num_processing_plants           int64
farm_area                     float64
farming_company                object
deidentified_location          object
temp_obs                      float64
cloudiness                    float64
wind_direction                float64
pressure_sea_level            float64
precipitation                 float64
wind_speed                    float64
month                           int64
day                             int64
hour                            int64
dtype: object

In [5]:
train_ing_w = train_final_merged[train_final_merged['ingredient_type'] == 'ing_w']

In [6]:
# Split the dataset into three equal parts
train_ing_w_splits = np.array_split(train_ing_w, 3)

# Access and use the first split as a separate dataset
train_w = train_ing_w_splits[0]


In [7]:
test_ing_w = test_final_merged[test_final_merged['ingredient_type'] == 'ing_w']

In [8]:
# Split the dataset into three equal parts
test_ing_w_splits = np.array_split(test_ing_w, 3)

# Access and use the first split as a separate dataset
test_w = test_ing_w_splits[0]

In [9]:
# keep a copy of test w we need it in creating submission file
import copy
tes = copy.deepcopy(test_w)

* farm_id is unique just like my id. my performance is measured based on my skills not based on my id right! same here the prediction of yield is depend on loaction and weather condtions on id. so let's drop it

In [10]:
# dropping the farm_id and num_processing pants from test and train datasets
train_w.drop(['farm_id','Unnamed: 0'], axis=1, inplace=True)
test_w.drop(['farm_id','Unnamed: 0'], axis=1, inplace=True)

In [11]:
# assigning object columns in category columns
cat_cols=['ingredient_type','deidentified_location',
          'operations_commencing_year','farming_company']
cat_cols

['ingredient_type',
 'deidentified_location',
 'operations_commencing_year',
 'farming_company']

In [12]:
# converting object columns to category columns
train_w[cat_cols] = train_w[cat_cols].astype("category")
test_w[cat_cols] = test_w[cat_cols].astype("category")

# Model Building and Predictions

In [13]:
# deviding the data into x and y
from sklearn.model_selection import train_test_split
X = train_w.drop(['yield'] , axis = 1)
y = train_w['yield']

In [14]:
# splitting the data into train and test in the ration of 70% & 30%
X_train, X_test, y_train, y_test =   train_test_split(X, y, test_size=0.30,random_state = 123)

In [15]:
# checking the shapes
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(2862975, 15) (1226990, 15)
(2862975,) (1226990,)


<div class="alert alert-block alert-info"><b> 
  Dummifing categorical features into numerical using get dummies
</b></div> 

In [16]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
X_train.deidentified_location = label.fit_transform(X_train.deidentified_location)
X_test.deidentified_location = label.transform(X_test.deidentified_location)
X_train.farming_company = label.fit_transform(X_train.farming_company)
X_test.farming_company = label.transform(X_test.farming_company)

In [17]:
## Convert Categorical Columns to numerical
X_train = pd.get_dummies(X_train, columns=['ingredient_type','operations_commencing_year'],drop_first=True)

X_test = pd.get_dummies(X_test, columns=['ingredient_type','operations_commencing_year'],drop_first=True)

In [18]:
# scaling the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
num_cols = ["farm_area", "wind_speed", "precipitation", "temp_obs",
            "pressure_sea_level","wind_direction",'num_processing_plants']

# scale on train
X_train[num_cols] =scaler.fit_transform(X_train[num_cols])

# scale on test
X_test[num_cols] = scaler.transform(X_test[num_cols])

In [19]:
# checking the x train data
X_train.head()

,num_processing_plants,farm_area,farming_company,deidentified_location,temp_obs,cloudiness,wind_direction,pressure_sea_level,precipitation,wind_speed,month,day,hour,operations_commencing_year_new,operations_commencing_year_old
146863,0.133333,0.123357,8,3,0.641484,4.0,0.500000,0.671745,0.002907,0.113514,2,28,23,0,1
2087311,0.466667,0.039678,14,4,0.394231,0.0,0.666667,0.754848,0.002907,0.081081,5,1,3,0,1
605829,0.133333,0.021570,14,3,0.725275,6.0,0.583333,0.623269,0.002907,0.167568,7,28,6,0,0
2122852,0.466667,0.031967,8,4,0.572802,2.0,0.138889,0.645429,0.002907,0.275676,5,24,16,1,0
3617023,0.133333,0.142015,8,1,0.755495,2.0,0.138889,0.548476,0.002907,0.113514,6,13,15,0,1


In [20]:
# checking the x test data
X_test.head()

,num_processing_plants,farm_area,farming_company,deidentified_location,temp_obs,cloudiness,wind_direction,pressure_sea_level,precipitation,wind_speed,month,day,hour,operations_commencing_year_new,operations_commencing_year_old
3365805,0.133333,0.140677,14,1,0.703297,2.0,0.805556,0.476454,0.002907,0.167568,5,8,22,0,0
5885964,0.133333,0.301549,8,0,0.534341,8.0,0.166667,0.508310,0.000000,0.167568,4,21,6,1,0
1178956,0.133333,0.140238,8,2,0.434066,2.0,0.583333,0.596953,0.002907,0.167568,2,11,20,0,0
1101228,0.133333,0.042940,8,2,0.434066,0.0,0.555556,0.596953,0.002907,0.248649,1,8,3,0,1
4706680,0.133333,0.035135,8,1,0.725275,2.0,0.861111,0.617729,0.002907,0.081081,11,14,20,1,0


In [21]:
# checking the data types x_train data
X_train.dtypes

num_processing_plants             float64
farm_area                         float64
farming_company                     int64
deidentified_location               int64
temp_obs                          float64
cloudiness                        float64
wind_direction                    float64
pressure_sea_level                float64
precipitation                     float64
wind_speed                        float64
month                               int64
day                                 int64
hour                                int64
operations_commencing_year_new      uint8
operations_commencing_year_old      uint8
dtype: object

# Model Building

<div class="alert alert-block alert-info"><b> 
  Linear Regression
</b>

In [22]:
# linear regression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train, y_train)
train_pred =reg.predict(X_train)
test_pred = reg.predict(X_test)
lr_train_pred = np.sqrt(mean_squared_error(y_train, train_pred))
lr_test_pred = np.sqrt(mean_squared_error(y_test, test_pred))
print('RMSE of Linear Regressor on train : ',lr_train_pred)
print('RMSE of Linear Regressor on test : ',lr_test_pred)

RMSE of Linear Regressor on train :  274.4271881698828
RMSE of Linear Regressor on test :  275.75809295975785


## Conclusion

<div class="alert alert-block alert-info"><b> 
   Linear regressor gave the best results
</b></div> 

## Lets work on test data

In [23]:
test_w.deidentified_location = label.fit_transform(test_w.deidentified_location)
test_w.farming_company = label.fit_transform(test_w.farming_company)

In [24]:
# converting categorical to numerical
test_w = pd.get_dummies(test_w, columns=['ingredient_type','operations_commencing_year'],drop_first=True)

In [25]:
# scale on test
test_w[num_cols] = scaler.transform(test_w[num_cols])

In [26]:
# selecting remaing columns except id column
fin = test_w.loc[:, test_w.columns != 'id']

In [27]:
# checking the final test dataset dimensions
fin.shape

(4199795, 15)

In [28]:
fin.dtypes

num_processing_plants             float64
farm_area                         float64
farming_company                     int64
deidentified_location               int64
temp_obs                          float64
cloudiness                        float64
wind_direction                    float64
pressure_sea_level                float64
precipitation                     float64
wind_speed                        float64
month                               int64
day                                 int64
hour                                int64
operations_commencing_year_new      uint8
operations_commencing_year_old      uint8
dtype: object

# Predictions

In [29]:
# predicting the test data with best model
test_pred = reg.predict(fin)

In [30]:
# creating a dataframe with id column and yield column
sub  = pd.DataFrame({'id': tes.id, 'yield': test_pred})

In [31]:
# checking the shape of the final dataframe 
sub.shape

(4199795, 2)

In [32]:
# checking the head of the dataframe
sub.head()

,id,yield
0,0,67.705353
1,1,62.482558
2,2,42.245791
3,3,62.539542
4,4,165.534569


In [33]:
# checking the last 5 rows of the dataframe
sub.tail()

,id,yield
6844736,28439126,123.476064
6844738,28439128,67.217960
6844739,28439129,99.693557
6844741,28439131,142.900828
6844742,28439132,223.101365


In [34]:
# creating submission file
sub.to_csv('submission_w1.csv', index = None)